In [ ]:
import numpy as np
from keras.datasets import imdb

In [ ]:
(train_x, train_y), (test_x, test_y) = imdb.load_data(num_words = 100)

In [ ]:
for i in range(len(train_x)):
    train_x[i] = [w for w in train_x[i] if w > 2]
for i in range(len(test_x)):
    test_x[i] = [w for w in test_x[i] if w > 2]

In [ ]:
i2w = {d[i]:i for i in d}
for w in train_x[0]:
    print(i2w[w-3], end = ' ')

this film was just story really the they and you just there is an and the from the as so i the there was a with this film the the film were great it was just so much that i the film as as it was for and would it to to and the was really at the it was so and you what they if you at a film it have been good and this was also to the that the of and they were just are out of the i because the that them all up are a for the film but are and be for what they have don't you the story was so because it was and was all that was with all 

In [ ]:
train_x.shape

(25000,)

In [ ]:
len(imdb.get_word_index())

1641221/1641221 [==============================] - 0s 0us/step


88584

In [ ]:
d = imdb.get_word_index()

In [ ]:
ck_d = [w for w in d.items()]
sorted(ck_d, key = lambda x:x[1])[0:5]

[('the', 1), ('and', 2), ('a', 3), ('of', 4), ('to', 5)]

In [ ]:
train_x[0]

In [ ]:
s_train_x = []
for i in range(len(train_x)):
    s_train_x.append([w for w in train_x[i] if w > 2])
s_train_x

In [ ]:
s_test_x = []
for i in range(len(train_x)):
    s_test_x.append([w for w in test_x[i] if w > 2])
s_test_x

In [ ]:
from keras.preprocessing import sequence
sequence


In [ ]:
import tensorflow as tf

In [ ]:
class RNN:
    def __init__(self, n_cells = 10, bs = 32, lr = 0.01):
        self.n_cells = n_cells
        self.w1x = None
        self.w1h = None
        self.b1 = None
        self.w2 = None
        self.b2 = None
        self.h = None
        self.losses = []
        self.val_losses = []
        self.bs = bs
        self.lr = lr

    def forpass(self, x):
        self.h = [np.zeros(x.shape[0].self.n_cells)]
        seq = np.swapaxes(x, 0.1)
        for x in seq:
            z1 = x@self.w1x + self.h[-1]@self.w1h + self.b1
            h = np.tanh(z1)
            self.h.append(h)
            z2 = h@self.w2 + self.b2
        return z2

    def backprop(self, x, err):
        m = len(x)
        w2_g = (self.h[-1].T@err) / m
        b2_g = np.sum(err) / m

        seq = np.swapaxes(x, 0.1)
        w1x_g = w1h_g = b1_g = 0
        c_err = err@self.w2.T*(1 - self.h[-1]**2)

        for x, h in zip(seq[::-1][:10], self.h[:-1][::-1][:10]):
            w1h_g += h.T@c_err
            w1x_g += x.T@c_err
            b1_g += np.sum(c_err, axis = 0)
            c_err = c_err@self.w1h.T*(1 - h**2)

        w1h_g /= m
        w1x_g /= m
        b1_g /= m

        return w1h_g, w1x_g, b1_g, w2_g, b2_g

    def sigmoid(self, z):
        z = np.clip(z, -100, None)
        a = 1/(1 + np.exp(-z))
        return a

    def init_w(self, n_f, n_c):
        orth = tf.initializers.Orthogonal()
        glorot = tf.initializers.GlorotUniform(0)

        self.w1h = orth((self.n_cells, self.n_cells)).numpy()
        self.wq1x = glorot((n_f, self.n_cells)).numpy()
        self.b1 = np.zeros(self.n_cells)

        self.w2 = glorot((self.n_cells, n_c)).numpy()
        self.b2 = np.zeros(n_c)

    def fit(self, x, y, epochs = 100, val_x = None, val_y = None):
        y = y.reshape(-1, 1)
        val_y = val_y.reshape(-1, 1)
        np.random.seed(42)
        self.init_w(x.shape[2], y.shape[1])
        for i in range(epochs):
            b_loss = []
            for x_b, y_b in self.gen_bs(x, y):
                a = self.training(x_b, y_b)
                a = np.clip(a, 1e-10, 1-1e-10)
                loss = np.mean(-(y_b*np.log(a) + (1 - y)*np.log(1 - a)))
                b_loss.append(loss)
            self.losses.append(np.mean(b_loss))
            self.update_val_loss(val_x, val_y)
            print(f"에포크 {i + 1} : train_loss{self.losses[-1]}")
    def gen_bs(self, x, y):
        l = len(x)
        bins = l//self.bs
        if i % self.bs:
            bins += 1
        idx = np.random.permutation(np.arange(len(x)))
        x = x[idx]
        y = y[idx]
        for i in range(bins):
            st = self.bs * i
            end = self.bs * (i + 1)
            yield x[st:end], y[st:end]

    def training(self, x, y):
        m = len(x)
        z = self.forpass(x)
        a = self.sigmoid(z)
        err = -(y - a)

        w1h_g, w1x_g, b1_g, w2_g, b2_g = self.backprop(x, err)

        self.w1h -= self.lr * w1h_g
        self.w1x -= self.lr * w1x_g
        self.b1 -= self.lr*b1_g
        self.w2 -= self.lr.w2_g
        self.b2 -= self.lr*b2_g

        return a

    def update_val_loss(self, val_x, val_y):
        z = self.forpass(val_x)
        a = self.sigmoid(z)
        a = np.clip(a, 1e-10, 1-1e-10)
        val_loss = np.mean(-(val_y * np.log(a) + (1 - val_y) * np.log(1-a)))
        self.val_losses.append(val_loss)

    def predict(self, x):
        z = self.forpass(x)
        a = self.sigmoid(z)
        return a > 0.5

    def score(self, x, y):
        py = self.predict(x)
        ty = y.reshape(-1, 1)
        np.mean(ty == py)

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
from keras.losses import binary_crossentropy

In [ ]:
import tensorflow as tf
import numpy as np
from keras.datasets import imdb
from keras.utils import pad_sequences
from keras.utils import to_categorical
(tr_x,tr_y),(tt_x,tt_y)=imdb.load_data(num_words=100) # 정수인코딩이 끝난 상태(문장이 숫자로 되어있음)
for i in range(len(tr_x)):
  tr_x[i]=[w for w in tr_x[i] if w > 2]
for i in range(len(tt_x)):
  tt_x[i]=[w for w in tt_x[i] if w > 2]
np.random.seed(42)
r_idx=np.random.permutation(25000)
s_tr_x=tr_x[r_idx[:20000]]
s_tr_y=tr_y[r_idx[:20000]]
s_tt_x=tr_x[r_idx[20000:]]
s_tt_y=tr_y[r_idx[20000:]]

# 패딩(단어의 길이를 맞춰줌) 패딩 : 데이터의 크기를 맞춰줌
p_tr_x=pad_sequences(s_tr_x,maxlen=100)
p_tt_x=pad_sequences(s_tt_x,maxlen=100)
s_tr_x=to_categorical(p_tr_x)
s_tt_x=to_categorical(p_tt_x)

In [ ]:
s_tr_x.shape

In [ ]:
np.unique(s_tr_y)

In [ ]:
m = Sequential(name="RNN")
m.add(SimpleRNN(100,input_shape=s_tr_x.shape[1:]))
m.add(Dense(1,activation='sigmoid'))
m.summary()

In [ ]:
s_tr_x.shape,s_tr_y.shape,s_tt_x.shape,s_tt_y.shape

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
ck=ModelCheckpoint("b_rnn_m.h5",save_best_only=True)
es=EarlyStopping(patience=5)
m.compile(loss='binary_crossentropy',metrics='acc')
hy=m.fit(s_tr_x,s_tr_y,validation_data=(s_tt_x,s_tt_y),epochs=100,batch_size=64,callbacks=[ck,es],verbose=2)